In [0]:
#Importing packages here 
import os 
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import re 
from nltk.metrics.distance import edit_distance
from sklearn.feature_extraction.text import TfidfVectorizer

In [13]:
#Mounting google drive 
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#Changing the working directory to this
#This cell needs to be executed once only otherwise it will enter the except block
try:
  os.chdir(os.getcwd()+"/drive/My Drive/IC/IC Project")
except:
    print("Already in the specified folder")


In [0]:
df = pd.read_csv("Data.csv",lineterminator='\n',index_col=0)

In [0]:
#Seperating the x,y 
x = df['Tweet']
y = df['Sentiment']

In [0]:
#Setting fixed random seed for consistent results
np.random.seed(0)

In [0]:
#Now need to do pre-processing steps
#Step 1: Case folding more popularly known as lowercasing :) 
x = x.str.lower()

In [19]:
#Step 2: Need to remove stopwords
stop = []
with open('stop.txt','r') as f:
    for word in tqdm(f):
      stop.append(word.strip())
#print(stop)
for word in tqdm(stop):
  x = x.replace(to_replace=r'\b{}\b'.format(word),value="",regex=True)

x = x.str.split()
#print(x)
       

102it [00:00, 307.19it/s]
100%|██████████| 102/102 [00:03<00:00, 26.18it/s]


In [0]:
x = x.drop([16904],axis=0) #this becomes nan so dropping it
y = y.drop([16904],axis=0) #also dropping this

In [0]:
#dropping the rows that become empty too after cleaning
drop = []
for word,i in zip(x,range(len(x))):
  if len(word)==0:  
      drop.append(i)
  

In [0]:
x = x.drop(drop,axis=0)
y = y.drop(drop,axis=0)

In [23]:
#Calculating edit distance first and removing words which have edit distance of 1 with the stop-words due to stop-words corpus being too small
distance = []
for word in tqdm(stop):
  dist = []
  for line in x:
    linelist = []
    for l in line[:]:
      d = edit_distance(word,l)
      if d==1:
        line.remove(l)
#Dropping the words which have length=0
result = x[x.apply(len)==0].index

y = y.drop(result,axis=0)
x = x.drop(result,axis=0)


100%|██████████| 102/102 [04:22<00:00,  2.57s/it]


In [0]:
#Step 4: Splitting into traning and testing data but first setting the random state to get consistent result everytime 
X_train,X_test,y_train,y_test = train_test_split(x,y)

In [0]:
#Step 5: Get count of words for the training data.
pseudoDict = {}
for values in X_train:
    for word in values:
        if word not in pseudoDict.keys():
            pseudoDict[word] = 1
        else:
            pseudoDict[word] = pseudoDict[word]+1

In [26]:
stopwords = []
threshold = 100
for val in pseudoDict:
    if pseudoDict[val]>threshold:
        stopwords.append(val)
print(stopwords)
#Removing these stopwords from the training corpus


['aaj', 'sath', 'sirf', 'khud', 'ek', 'pakistani', 'lekin', 'bat', 'pakistan', '?', 'karachi', '1', 'karte', 'nawaz', 'sharif', 'tak', 'shadi', 'baad', 'imran', 'aap', 'apna', 'sir', 'm', 'meri', 'taraf', '.', 'bohat', 'kaam', 'shamil', 'hussain', 'and', 'mera', 'key', 'bad', 'apni', 'film', 'unhon', 'apne', 'of', 'hone', 'bari', ',', 'diya', 'lahore', ':p', 'good', 'mere', '..', 'naam', 'jin', 'allah', 'bilkul', 'ghar', 'ameen', 'zindagi', 'plz', 'duniya', 'tarah', 'nazar', 'app', 'drama', 'hasil', 'hahaha', '...', '3', 'baat', '2', 'waqt', 'aunty', 'inhe', 'mien', 'agar', 'mujhe', ':d', 'inho', 'apny', 'dua', '😂', 'team', 'mulk', 'saal', 'dekh', 'logon']


In [27]:
#Step 7: Words that are present too often are also stop words.Need to remove the words after seeing what words are those
for word in tqdm(stopwords):
  for linelist in X_train:
    for token in linelist[:]:
      if token in stopwords:
        linelist.remove(token)

result = X_train[X_train.apply(len)==0].index
y_train = y_train.drop(result,axis=0)
X_train = X_train.drop(result,axis=0)
X_train = X_train.str.join(" ")
X_test = X_test.str.join(" ")

100%|██████████| 83/83 [00:09<00:00,  9.17it/s]


In [28]:
X_test

6055                          circuit ver-o board banaya .
7311     aap fitri kahaniyon taraf ziyada mutawajja hon...
19382                          . miss rubina chkma dia :-d
12217       sale apna deen eman bech dete paiso lanat amir
5061                     hahahaha..ye aap porii summary 😅😅
                               ...                        
18024                        pallz tell police test result
17773           shukr khudi ruk pehle. ktne raaz khole ...
6236     unhon apnay pehlay dora-e-hukumat 1989 islamab...
937                               aadmi aap majboor karen.
2506                    wajah adakari bare dilchaspi paida
Name: Tweet, Length: 4962, dtype: object

In [29]:
X_train

6103             khwaroon badshah salamat bachay class li.
20044    bol itni khubsurat sarkey hmarey bacho dekhna ...
13440    cream itne bare nikalti 10g mushkil 10days chalti
15726                                           hdiya kasy
10463    saktin farigh aurtein😂😂😂 gor karein😆😆 #fazool_...
                               ...                        
13401                                               bekaar
20031                            shrkiyan leader supporter
10110                     tabah politicians corrupt loghon
11068             nazimabad mujahid colony firing 8 zakhmi
2743                            javed miandad career urroj
Name: Tweet, Length: 14651, dtype: object

In [0]:
#Step 8: Create a tf-idf representation of the training corpus 
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train) 
X_test_vec = vectorizer.transform(X_test)

In [0]:
X_train_vec = X_train_vec.toarray()

In [0]:
X_test_vec = X_test_vec.toarray()

In [0]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
y_train = labelencoder.fit_transform(y_train)
y_test = labelencoder.transform(y_test)

In [34]:
from keras.utils import to_categorical
y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)

Using TensorFlow backend.


In [35]:
import tensorflow as tf
from tensorflow import keras
embedding_size = X_train_vec.shape[1]
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(embedding_size,)),#donot change(input layer)
    keras.layers.Dense(300, activation='relu'),#(hidden layer)
    keras.layers.Dense(50, activation='relu'),#(hidden layer)
    keras.layers.Dense(4)#donot change
])
model.compile(optimizer='adam',
              loss=["categorical_crossentropy"],
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 25932)             0         
_________________________________________________________________
dense (Dense)                (None, 300)               7779900   
_________________________________________________________________
dense_1 (Dense)              (None, 50)                15050     
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 204       
Total params: 7,795,154
Trainable params: 7,795,154
Non-trainable params: 0
_________________________________________________________________


In [0]:
train_X = X_train_vec
train_Y = y_train_hot

In [37]:
model.fit(train_X,train_Y, epochs=25, batch_size=32,
               verbose=1,shuffle=True,validation_split=0.4)

Epoch 1/25
275/275 [==============================] - 3s 11ms/step - loss: 5.7695 - accuracy: 0.3090 - val_loss: 5.8406 - val_accuracy: 0.3044
Epoch 2/25
275/275 [==============================] - 2s 8ms/step - loss: 6.0501 - accuracy: 0.3002 - val_loss: 5.8406 - val_accuracy: 0.3044
Epoch 3/25
275/275 [==============================] - 2s 8ms/step - loss: 6.0501 - accuracy: 0.3002 - val_loss: 5.8406 - val_accuracy: 0.3044
Epoch 4/25
275/275 [==============================] - 2s 8ms/step - loss: 6.0493 - accuracy: 0.3002 - val_loss: 5.8406 - val_accuracy: 0.3044
Epoch 5/25
275/275 [==============================] - 2s 8ms/step - loss: 6.0534 - accuracy: 0.3002 - val_loss: 5.8406 - val_accuracy: 0.3044
Epoch 6/25
275/275 [==============================] - 2s 8ms/step - loss: 6.0518 - accuracy: 0.3002 - val_loss: 5.8406 - val_accuracy: 0.3044
Epoch 7/25
275/275 [==============================] - 2s 8ms/step - loss: 6.0509 - accuracy: 0.3002 - val_loss: 5.8406 - val_accuracy: 0.3044
Epoch

In [38]:
predictions1 = model.predict(X_test_vec)
test_Y = y_test_hot
predictions = np.argmax(predictions1,axis=-1)
predictions

array([3, 3, 3, ..., 3, 3, 3])

In [39]:
mapping = dict(zip(labelencoder.classes_, range(len(labelencoder.classes_))))
mapping

{'Neative': 0, 'Negative': 1, 'Neutral': 2, 'Positive': 3}

In [40]:
loss, acc = model.evaluate(X_test_vec, y_test_hot,
                            batch_size=32)
print('Loss score:', loss)
print('Test accuracy:', acc)

156/156 [==============================] - 0s 3ms/step - loss: 5.6697 - accuracy: 0.3007
Loss score: 5.669748783111572
Test accuracy: 0.30068519711494446
